In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Introduction to Pandas. Part IV

Contents

- [Applying a function to a pandas Series or DataFrame](#1.-Applying-a-function-to-a-pandas-Series-or-DataFrame)
- [Using groupby](#2.-Using-groupby)
- [Unstacking](#3.-Unstacking)
- [Pivot tables and cross tabulations](#4.-Pivot-tables-and-cross-tabulations)
- [Rolling and resampling](#5.-Rolling-and-resampling)

## 1. Applying a function to a pandas Series or DataFrame

- [The map method](#1.1.-The-map-method)
- [The apply method](#1.2.-The-apply-method)
- [The applymap method](#1.3.-The-applymap-method)

In [ ]:
url = "https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/titanic.csv"
titanic = pd.read_csv(url)
titanic.head()

### 1.1. The map method

map is a Series method.

map allows you to map an existing value of a series to a different set of values.

In [ ]:
my_map = {'female':1,'male':0}
my_map

In [ ]:
# map 'female' to 0 and 'male' to 1
titanic['Sex_num'] = titanic.Sex.map(my_map)

In [ ]:
titanic.Sex_num

In [ ]:
titanic.loc[0:4,['Sex','Sex_num']]

### 1.2. The apply method

apply is both a Series method and a DataFrame method

- [Apply as a Series method](#1.2.1.-Apply-as-a-Series-method)
- [Apply as a DataFrame method](#1.2.2.-Apply-as-a-DataFrame-method)

#### 1.2.1. Apply as a Series method

apply applies a function to each element of the Series

**Remark:** **map** can be substituted for **apply** in many cases, but **apply** is more flexible and thus is recommended

**Example 1:** calculate the length of the strings in the 'Name' column

In [ ]:
titanic['Name_length'] = titanic.Name.apply(len) # apply Python len function

In [ ]:
titanic.loc[0:4,['Name','Name_length']]

In [ ]:
titanic.Name.map(len)

**Example 2:** round up each element in the 'Fare' Series to the next integer

In [ ]:
import numpy as np
titanic['Fare_ceil'] = titanic.Fare.apply(np.ceil) # apply Numpy ceiling function

In [ ]:
titanic.loc[0:4,['Fare','Fare_ceil']]

**Example 3:** Extract the last name of each person into its own column

In [ ]:
def get_last_name(x):
    return x.split(',')[0]

In [ ]:
titanic['Last_name'] = titanic.Name.apply(get_last_name)

In [ ]:
titanic.loc[0:4,['Name','Last_name']]

In [ ]:
titanic.Name.map(get_last_name)

In [ ]:
# alternatively, use a lambda function
titanic['Last_name'] = titanic.Name.apply(lambda x:x.split(',')[0])

In [ ]:
titanic.loc[0:4,['Name','Last_name']]

#### 1.2.2. Apply as a DataFrame method

apply applies a function along either axis of the DataFrame

In [ ]:
# read a dataset of alcohol consumption into a DataFrame
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/drinks.csv'
drinks = pd.read_csv(url)
drinks.head()

In [ ]:
# select a subset of the DataFrame to work with
sub_drinks = drinks.loc[:,'beer_servings':'wine_servings']
sub_drinks.head()

In [ ]:
# apply the 'max' function along axis 0 to calculate the maximum value in each column
sub_drinks.apply(max, axis=0) # Python max function along rows

In [ ]:
# apply the 'max' function along axis 1 to calculate the maximum value in each row
sub_drinks.apply(max, axis=1) # Python max function along columns

In [ ]:
# use 'np.argmax' to calculate which column has the maximum value for each row
sub_drinks.apply(np.argmax,axis=1)

In [ ]:
'if you want to know in which column is the maximum'
sub_drinks.apply(np.argmax,axis=1).map({0:'beer_servings',
                                        1:'spirit_servings',
                                        2:'wine_servings'})

### 1.3. The applymap method

applymap is a DataFrame method. It applies a function to everyelement of the DataFrame

In [ ]:
sub_drinks.head(5)

In [ ]:
# convert every DataFrame element into a float
sub_drinks.applymap(float).head(5)

In [ ]:
# overwrite the existing DataFrame columns
sub_drinks.loc[:, 'beer_servings':'wine_servings'] = sub_drinks.loc[:, 'beer_servings':'wine_servings'].applymap(float)
sub_drinks.head()

## 2. Using groupby

In [ ]:
# reload the drinks dataset
drinks = pd.read_csv('https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/drinks.csv')
drinks.head()

In [ ]:
# calculate the average beer servings across the entire dataset
drinks.beer_servings.mean()

In [ ]:
# calculate the average beer servings just for countries in Africa
drinks[drinks.continent=='Africa'].beer_servings.mean()

In [ ]:
# calculate the mean beer servings for each continent
drinks.groupby('continent').beer_servings.mean()

In [ ]:
# bar plot of the above DataFrame 
drinks.groupby('continent').beer_servings.mean().plot(kind='bar')

In [ ]:
# other aggregation functions (such as 'max','min','count','std', etc) can also be used with groupby
drinks.groupby('continent').beer_servings.max()

In [ ]:
# you can use your own aggregation function
def peak_to_peak(group):
    return group.max()-group.min()
drinks.groupby('continent').beer_servings.apply(peak_to_peak)

In [ ]:
# multiple aggregation functions can be applied simultaneously
drinks.groupby('continent').beer_servings.agg(['count', 'mean', 'min', 'max','std'])

In [ ]:
drinks.groupby('continent').beer_servings.agg(['count', 'mean', 'min', 'max','std',peak_to_peak])

In [ ]:
# specifying a column to which the aggregation function should be applied is not required
drinks.groupby('continent').mean()

In [ ]:
# side-by-side bar plot of the DataFrame directly above
drinks.groupby('continent').mean().plot(kind='bar')

## 3. Unstacking

In [ ]:
# load tips dataset
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/tips.csv'
tips = pd.read_csv(url)
tips.head()

In [ ]:
# add tip percentage of total bill
tips['tip_pct'] = 100*tips.tip/tips.total_bill
tips.head()

In [ ]:
# aggregate tip_pct by day and smoker
tips.groupby(['day','smoker']).tip_pct.mean() # DataFrame with a multi-index

In [ ]:
tips.groupby(['day','smoker']).tip_pct.mean().unstack(level=1)

In [ ]:
tips.groupby(['day','smoker']).tip_pct.mean().unstack(level=1).plot(kind='bar')

In [ ]:
tips.groupby(['day','smoker']).tip_pct.mean().unstack(level=0)

In [ ]:
tips.groupby(['day','smoker']).tip_pct.mean().unstack(level=0).plot(kind='bar')

## 4. Pivot tables and cross-tabulations

- [Pivot tables](#4.1.-Pivot-tables)
- [Cross-Tabulations](#4.2.-Cross-Tabulations)

### 4.1. Pivot tables

A *pivot table* is a data summarization tool.
It aggregates a table of data by one or more keys, arranging the data in a rectangle.

In [ ]:
tips.head()

Aggregate 'tip_pct' by columns 'day' and 'smoker':

In [ ]:
# using groupby
tips.groupby(['day','smoker']).tip_pct.mean()

In [ ]:
# using pivot_table
tips.pivot_table('tip_pct', index='day', columns = 'smoker', aggfunc='mean') # default aggregation function = mean

In [ ]:
tips.pivot_table('tip_pct', index='day', columns = 'smoker') 

Aggregate 'tip_pct' and 'size' by 'time', 'day' and 'smoker'

In [ ]:
# using groupby
tips.groupby(['time','day','smoker'])[['tip_pct','size']].mean()

In [ ]:
# using pivot_table (arranging 'time' and 'day' on the rows)
tips.pivot_table(['tip_pct','size'], index=['time','day'], columns = 'smoker', aggfunc='mean')

In [ ]:
# use 'count' as the aggregation function
tips.pivot_table(['tip_pct'], index=['time','day'], columns = 'smoker', aggfunc='count') 

In [ ]:
tips.pivot_table(['tip_pct'], index=['time','day'], columns = 'smoker', aggfunc=len, fill_value=0)

We can augment this table to include partial totals by passing margins=True

In [ ]:
tips.pivot_table('tip_pct', index=['time','day'], columns = 'smoker', aggfunc='count', fill_value=0, margins=True)

In [ ]:
tips.pivot_table('tip_pct',index='day',columns = 'smoker', aggfunc='count', margins=True)

### 4.2. Cross-Tabulations

A *cross-tabulation* is a special case of pivot table that computes frequencies.

In [ ]:
pd.crosstab(index=tips.day, columns = tips.smoker)

In [ ]:
pd.crosstab(index=tips.day,columns=tips.smoker,margins=True)

In [ ]:
pd.crosstab(index=tips.day, columns = tips.smoker, normalize='index')

In [ ]:
pd.crosstab(index=tips.day, columns = tips.smoker, normalize='columns')

In [ ]:
pd.crosstab(index=[tips.time,tips.day], columns = tips.smoker,  normalize='index')

## 5. Resampling and rolling

 - [Resampling](#5.1.-Resampling)
 - [Rolling](#5.2.-Rolling)

Let us take a look at bicycle counts on Seattle's Fremont Bridge

In [ ]:
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/Fremont.csv'
fremont = pd.read_csv(url)
fremont.head()

The Fremont Bridge Bicycle Counter began operation in October 2012 and records the number of bikes that cross the bridge using the pedestrian/bicycle pathways.
The bicycle counter has sensors on the east and west sidewalks of the bridge.

<table><tr>
<td> <img src="fremont.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="fremont2.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

In [ ]:
fremont['Date'] = pd.to_datetime(fremont.Date)

In [ ]:
fremont.set_index('Date', inplace=True)

In [ ]:
fremont.head()

In [ ]:
# Better way:
fremont = pd.read_csv(url, index_col='Date', parse_dates=True)
fremont.head()

For convenience, we'll further process this dataset by shortening the column names:

In [ ]:
fremont.columns = ['Total', 'East', 'West']

In [ ]:
fremont.columns

Let's plot the raw data:

In [ ]:
fremont.plot(figsize=(12,7)) 

The ~25,000 hourly samples are far too dense for use to make much sense of.
We can gain more insight by *resampling* the data to a courser grid.

### 5.1. Resampling

Resampling refers to the process of converting a time series from one frequency to another. 

In [ ]:
# resample by day
fremont.resample('D').sum().plot(figsize = (12,5)) # D = dayly
plt.title('Daily bicycle count',fontsize=20)

In [ ]:
# resample by week
fremont.resample('W').sum().plot(figsize=(12,5)) # W : weekly
plt.title('Weekly bicycle count',fontsize=20)

This shows us some interesting seasonal trends: people bicycle more in the summer than in the winter.

### 5.2 Rolling

Rolling means (or moving averages) are generally used to smooth out short-term fluctuations in time series data and highlight long-term trends

In [ ]:
fremont.resample('D').sum().plot(figsize=(12,5)) # data is very spiky

In [ ]:
fremont.resample('D').sum().rolling(10).mean().plot(figsize=(12,5)) 

The expression 'rolling(10)' creates an object that enables grouping over a 10-day sliding window.
So here we have the 10-day moving window average of Fremont traffic.

**Bonus: digging into the data**

While the smoothed data views are useful to get an idea of the general trend in the data, they hide much of the interesting structure.

Let's plot the hourly counts of each day

In [ ]:
pivoted = fremont.pivot_table('Total', index=fremont.index.time, columns = fremont.index.date)
pivoted.head()

In [ ]:
pivoted.plot(legend=False,alpha=0.01, figsize=(12,5))

We can see two trends: One trend that peaks in the morning and in the evening (weekday trend?), and another trend that peaks at noon (weekend trend?).

Let us plot the total count by day of the week.

In [ ]:
pivoted_daily = fremont.pivot_table('Total', index = fremont.index.time, columns = fremont.index.dayofweek)
pivoted_daily.head()

In [ ]:
pivoted_daily.plot(figsize=(12,5))

This shows a strong distinction between weekday and weekend totals